# ML Cup Event #3

In [ ]:
import pandas as pd
import numpy as np
import imblearn
from imblearn.over_sampling import SMOTE

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

In [ ]:
import os
os.getcwd()

In [ ]:
df = pd.read_csv("diabetes_train.csv")

Id_col = 'Id'
target_col = 'diabetes'
df.info()
df.head()
df.describe()
df.isnull().sum()

In [26]:
#Replacing missing  values
df['BMI'].fillna(df['BMI'].mean(), inplace = True)
df['serum_insulin'].fillna(df['serum_insulin'].mean(), inplace = True)
df['DBP'].fillna(df['DBP'].mean(), inplace = True)



AttributeError: 'DataFrame' object has no attribute 'serum_insulin'

In [ ]:
df.info()

In [ ]:
def set_bmi(row):
    if row["BMI"] < 18.5:
        return "Under"
    elif row["BMI"] >= 18.5 and row["BMI"] <= 24.9:
        return "Healthy"
    elif row["BMI"] >= 25 and row["BMI"] <= 29.9:
        return "Over"
    elif row["BMI"] >= 30:
        return "Obese"

In [ ]:
df = df.assign(BMI_DESC=df.apply(set_bmi, axis=1))

df.head()

In [ ]:
def set_insulin(row):
    if row["serum_insulin"] >= 16 and row["serum_insulin"] <= 166:
        return "Normal"
    else:
        return "Abnormal"


In [ ]:
df = df.assign(INSULIN_DESC=df.apply(set_insulin, axis=1))

df.head()

In [ ]:
z=pd.get_dummies(df.BMI_DESC, prefix='BMI_DESC')
y = pd.get_dummies(df.INSULIN_DESC, prefix='INSULIN_DESC')
df = pd.concat([df, y,z], axis=1)
df=df.drop(['BMI_DESC','INSULIN_DESC','BMI','serum_insulin'], axis=1)
df

In [ ]:
#Normalize Data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

features = ['num_times_pregnant','plasma_glucose','DBP','triceps_skin','age']
df[features] = scaler.fit_transform(df[features])

In [13]:
from sklearn.model_selection import train_test_split

X = df.drop([Id_col, target_col], axis=1)
y = df[target_col]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
oversample = SMOTE()
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
X.info()
X.shape
X.head()

X_train.info()
X_train.shape
X_train.head()

# Build Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=4)
clf.fit(X_train, y_train)



In [25]:
from sklearn.model_selection import GridSearchCV

base_clf = RandomForestClassifier(class_weight=None, random_state=42)
parameters_grid = {'criterion': ('gini', 'entropy'),
                  'max_depth': [2, 3, 4, 5, 6],
                  'max_leaf_nodes': [None, 5, 10, 50, 100],
                  'min_impurity_decrease': [0, 0.1, 0.2],
                  'min_samples_leaf': [1, 10, 50],
                  'min_samples_split': [2, 10, 50]}
clf_gs = GridSearchCV(base_clf, param_grid=parameters_grid, scoring="f1", n_jobs=3, cv=5, return_train_score=True)
%time clf_gs.fit(X_train, y_train)

Wall time: 15min 24s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=42), n_jobs=3,
             param_grid={'criterion': ('gini', 'entropy'),
                         'max_depth': [2, 3, 4, 5, 6],
                         'max_leaf_nodes': [None, 5, 10, 50, 100],
                         'min_impurity_decrease': [0, 0.1, 0.2],
                         'min_samples_leaf': [1, 10, 50],
                         'min_samples_split': [2, 10, 50]},
             return_train_score=True, scoring='f1')

In [ ]:
clf_gs.best_params_

In [23]:
from sklearn.ensemble import RandomForestClassifier
Model=RandomForestClassifier()
Model.fit(X_train,y_train)


RandomForestClassifier()

In [ ]:
from xgboost import XGBClassifier

XGB = XGBClassifier(max_depth=3)
XGB.fit(X_train, y_train)

In [16]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(solver='lbfgs',class_weight='balanced', max_iter=50000)
log_reg.fit(X_train, y_train)



LogisticRegression(class_weight='balanced', max_iter=50000)

# Estimate Model Performance

In [ ]:
from sklearn.metrics import confusion_matrix

#pred_val = clf.predict(X_val)
#pred_val = Model.predict(X_val)
#pred_val = XGB.predict(X_val)
#pred_val = DTC.predict(X_val)
pred_val=log_reg.predict(X_val)
#pred_val = clf_gs.predict(X_val)


confusion_matrix(y_val, pred_val)

In [ ]:
from sklearn.metrics import classification_report, f1_score

print("F1 Score = {:.2f}".format(f1_score(y_val, pred_val)))
print()
print(classification_report(y_val, pred_val))

# Create Submission File on Test Data

In [ ]:
test_df = pd.read_csv('diabetes_test.csv')



In [ ]:
test_df

In [ ]:
test_df = test_df.assign(BMI_DESC=test_df.apply(set_bmi, axis=1))
test_df = test_df.assign(INSULIN_DESC=test_df.apply(set_insulin, axis=1))
z=pd.get_dummies(test_df.BMI_DESC, prefix='BMI_DESC')
y = pd.get_dummies(test_df.INSULIN_DESC, prefix='INSULIN_DESC')
test_df = pd.concat([test_df, y,z], axis=1)
test_df=test_df.drop(['BMI_DESC','INSULIN_DESC','BMI','serum_insulin'], axis=1)
test_df[features] = scaler.fit_transform(test_df[features])
test_df

In [ ]:
# Make sure to transform/create the features the same way you did for training
# For now, nothing to do

X_test = test_df.drop([Id_col], axis=1)

# Use your model to make predictions
pred_test = log_reg.predict(X_test)

# We will look at the predicted prices to ensure we have something sensible.
print(pred_test)

my_submission = pd.DataFrame({'Id': test_df.Id, 'Predicted': pred_test})

my_submission.head()

# you could use any filename. We choose submission here
my_submission.to_csv('my_submission_7.csv', index=False)